# Namespaces Lab

In this lab, we will explore how namespaces are presented in the Linux shell and how you can interact with namespaces using some simple commands. During the lab we will explore how you can interact with both the Network and Mount namespaces, which are commonly used by containers as an isolation mechanism to separate workloads running a single host.

To start, lets explore a simple example of how namespaces work. Lets look at the network namespace.

Here are the interfaces on our host:

In [ ]:
ip link

Now, Lets use unshare to create a new namespace and run the same command to see the interfaces in this namespace:

In [ ]:
sudo unshare -n ip link

Note, we have a loopback interface, but the parameters attached to this interface are different to loopback on the host OS. This is a separate logical interface which has been facilitated by namespaces.

## Docker namespace utilization demo

Now lets take a look at how docker uses namespaces to separate containers, and explore how we can interact with this namespace from the host OS. 

To start, lets run a redis container:

In [ ]:
docker pull redis

In [ ]:
docker run --name redis -d redis

For the purposes of this lab, we will need to know the PID the Redis container is running under from the Host OS. We can extract this with the following shell command:

In [ ]:
REDIS_PID=$(ps ax | grep redis | grep -v grep | awk '{print $1}')
echo $REDIS_PID

From the Host OS, we can see the redis process.. even though redis is in a separate namespace. This is because the PID namespace is a little bit different and is visible to the host OS to allow us to see all running processes within the kernel. 

Next, we can find the network namespace id for our process by exploring the /proc filesystem with the readlink command:

In [ ]:
sudo readlink /proc/$REDIS_PID/ns/net

Lets look deeper and examine the networking configuration of the Redis container we've started up. First, lets pull some information from Docker to gather the IP address recorded for the container: 

In [ ]:
CONTAINER_IP=$(docker inspect redis | jq .[0].NetworkSettings.IPAddress | sed -e s/\"//g)
echo $CONTAINER_IP

Now to cross check, lets enter the namespace created for the redis-server process (aka the Docker container), and run the ifconfig command. We should see the same IP address as recorded by Docker:

In [ ]:
sudo nsenter -t $REDIS_PID -n ifconfig

As docker uses a bridge network by default, I can telnet to the redis-server from the host OS

In [ ]:
(printf "PING\r\n";) | nc $CONTAINER_IP 6379 

Now lets explore how we can interact with the container namespace from the host OS.

To do this, lets launch a nginx process from the Host OS, in the redis container namespace. 

Now, nginx does not ship in the docker container for redis, and the redis container is based on Ubuntu Linux, but we do have nginx installed on our Host OS, running Amazon Linux. 

In the next example, we namespace for our redis server, but execute nginx from our host OS, which is running Amazon Linux:

In [ ]:
sudo nsenter -t $REDIS_PID -n nginx

Now, to validate the nginx process is not running in the host OS network namespace, i can try to connect to localhost:

In [ ]:
curl http://localhost

However, if i connect to the IP associated with the redis container network namespace:

In [ ]:
curl http://$CONTAINER_IP

You can see this is the Amazon Linux distribution of Redis. 

I'm running a process in the same networking space as my container, but not in the container. It's sharing the networking space, but it's not running under Ubuntu which is what the container is running

Interacting with namespaces on a host can be a useful tool for troubleshooting. By using commands like the above above, we can interact with the networking stack the docker container is running in. In turn, we can use this for troubleshooting to perform tests and rule out configuration problems in the docker container without needing to rebuild the container and redeploy. 

## Exploring the mount namespace

Lets now look at this from the opposite direction: We don't have redis installed on the Amazon Linux host OS, but we do have this as part of our redis container that is running. Lets explore accessing the mount namespace from the Host OS. 

First, lets show there is no redis-server installed or running on localhost:

In [ ]:
which redis-server

In [ ]:
(printf "PING\r\n";) | nc localhost 6379 

Next, we can enter the mount namespace for our redis container, and run this binary on the host OS:

In [ ]:
sudo nsenter -t $REDIS_PID -m /usr/local/bin/redis-server &

Now we can test the redis server in the host OS network namespace:

In [ ]:
(printf "PING\r\n";) | nc localhost 6379 

We can also see we have two instances of redis running:

In [ ]:
ps ax | grep redis | grep -v grep

### Lab Cleanup

After running this demo, use the following commands to clean up the lab and stop any running redis-service processes:

In [ ]:
docker ps

In [ ]:
docker stop redis

In [ ]:
ps -ef | grep redis

Use the kill command in the next section to kill the nsenter and redis-server process IDs

In [ ]:
#sudo kill -9 

In [ ]:
docker rm redis